In [135]:
import fastcore.transform
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fastai.tabular.all import *
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

device = "cuda" if torch.cuda.is_available() else "cpu"
device

[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'cuda'

In [136]:
data = pd.read_csv("../data/video_data.csv")
data.head()

,Video Title,Video Like Count,Video View Count,Video Comment Count
0,Coding for 1 Month Versus 1 Year #shorts #coding,220649,4601638,2432
1,what 10 years of coding looks like…,1252936,21172887,10404
2,What is Coding?,23012,1570351,800
3,Programming vs Coding - What&#39;s the difference?,55928,1942385,1777
4,How to ACTUALLY learn to code... 7 Roadmaps for 2023,120079,1918790,2861


In [137]:
data['Video Like Count'] = data['Video Like Count'].fillna(0).astype(int)
data['Video View Count'] = data['Video View Count'].fillna(0).astype(int)
data['Video Comment Count'] = data['Video Comment Count'].fillna(0).astype(int)

In [138]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a text encoder
text_encoder = CountVectorizer(max_features=1000)  # Adjust max_features as needed
text_features = text_encoder.fit_transform(data['Video Title'])

# Convert to DataFrame
text_df = pd.DataFrame(text_features.toarray(), columns=text_encoder.get_feature_names_out())

# Concatenate with original DataFrame
df = pd.concat([data, text_df], axis=1)

# Now remove the original 'Video Title' column from cat_names
cat_names = []  # or other categorical columns if any
cont_names = ['Video Like Count', 'Video View Count', 'Video Comment Count'] + list(text_df.columns)

for col in cont_names:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.fillna(0)

In [139]:
dep_var = 'Video View Count'
from fastai.tabular.all import *

procs = [Categorify, FillMissing, Normalize]

dls = TabularDataLoaders.from_df(df, 
                                 dep_var, 
                                 cat_names=cat_names, 
                                 cont_names=cont_names, 
                                 procs=procs,
                                 y_block=RegressionBlock(),
                                 splits=RandomSplitter(valid_pct=0.2)(range_of(df)))

print("DataLoaders created successfully")

DataLoaders created successfully


In [141]:
y_min = float(df[dep_var].min())
y_max = float(df[dep_var].max())

dls.c = 1
learn = tabular_learner(dls, 
                        layers=[200,100],  # you can adjust these
                        metrics=accuracy,
                        y_range=(y_min, y_max))

print("Learner created successfully")

Learner created successfully


In [142]:
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time


/home/user/miniconda3/lib/python3.12/site-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")
/home/user/miniconda3/lib/python3.12/site-packages/torch/_tensor.py:1418: UserWarning: Using a target size (torch.Size([0])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  ret = func(*args, **kwargs)


RuntimeError: The size of tensor a (10) must match the size of tensor b (0) at non-singleton dimension 0